# Lab | Making predictions with logistic regression


In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to predict if a film will have more monthly rentals in July than in June. Create a model to predict it.

Instructions

### 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.


In [113]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  
password = getpass.getpass()

········


In [114]:
import numpy as np

In [115]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [116]:
query = 'select * from film'
data_film = pd.read_sql_query(query, engine)
data_film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [117]:
query = 'select * from rental'
data_rental = pd.read_sql_query(query, engine)
data_rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [118]:
query = 'select * from inventory'
data_inventory = pd.read_sql_query(query, engine)
data_inventory.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [119]:
query = 'select r.rental_id, r.rental_date, i.film_id \
from rental as r \
left join inventory as i \
on r.inventory_id = i.inventory_id'
data = pd.read_sql_query(query, engine)
data.head()

,rental_id,rental_date,film_id
0,1,2005-05-24 22:53:30,80
1,2,2005-05-24 22:54:33,333
2,3,2005-05-24 23:03:39,373
3,4,2005-05-24 23:04:41,535
4,5,2005-05-24 23:05:21,450


In [120]:
query = 'select r.rental_id, r.rental_date, YEAR(r.rental_date) AS year, MONTH(r.rental_date) AS month, i.film_id, f.title, f.rental_duration, f.rental_rate, f.special_features \
from rental as r \
left join inventory as i \
on r.inventory_id = i.inventory_id \
left join film as f \
on i.film_id = f.film_id \
having year = 2005'

data = pd.read_sql_query(query, engine)
data.head()

,rental_id,rental_date,year,month,film_id,title,rental_duration,rental_rate,special_features
0,1,2005-05-24 22:53:30,2005,5,80,BLANKET BEVERLY,7,2.99,Trailers
1,2,2005-05-24 22:54:33,2005,5,333,FREAKY POCUS,7,2.99,"Trailers,Behind the Scenes"
2,3,2005-05-24 23:03:39,2005,5,373,GRADUATE LORD,7,2.99,"Trailers,Behind the Scenes"
3,4,2005-05-24 23:04:41,2005,5,535,LOVE SUICIDES,6,0.99,"Trailers,Behind the Scenes"
4,5,2005-05-24 23:05:21,2005,5,450,IDOLS SNATCHERS,5,2.99,Trailers


### 2. Create a query to get the total amount of rentals in June for each film. 


In [121]:
query = 'select i.film_id ,YEAR(r.rental_date) AS year, MONTH(r.rental_date) AS month, count(i.film_id) as total_rent, f.title, f.rental_duration, f.rental_rate, f.special_features \
from rental as r \
left join inventory as i \
on r.inventory_id = i.inventory_id \
left join film as f \
on i.film_id = f.film_id \
group by i.film_id, year, month \
having year = 2005 and month = 6 \
order by film_id;'

data_june = pd.read_sql_query(query, engine)
data_june.head()


,film_id,year,month,total_rent,title,rental_duration,rental_rate,special_features
0,1,2005,6,3,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,3,2005,6,1,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
2,4,2005,6,2,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
3,5,2005,6,2,AFRICAN EGG,6,2.99,Deleted Scenes
4,6,2005,6,3,AGENT TRUMAN,3,2.99,Deleted Scenes


In [122]:
data_june.shape

(900, 8)

### 3. Do the same with July.

In [123]:
query = 'select i.film_id ,YEAR(r.rental_date) AS year, MONTH(r.rental_date) AS month, count(i.film_id) as total_rent, f.title, f.rental_duration, f.rental_rate, f.special_features \
from rental as r \
left join inventory as i \
on r.inventory_id = i.inventory_id \
left join film as f \
on i.film_id = f.film_id \
group by i.film_id, year, month \
having year = 2005 and month = 7 \
order by film_id;'

data_july = pd.read_sql_query(query, engine)
data_july.head()

,film_id,year,month,total_rent,title,rental_duration,rental_rate,special_features
0,1,2005,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,2,2005,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes"
2,3,2005,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
3,4,2005,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
4,5,2005,7,5,AFRICAN EGG,6,2.99,Deleted Scenes


In [124]:
data_july.shape

(958, 8)

In [125]:
# problem of shape :(

### 4. Create a new column containing (Yes/No) for each film whether or not the number of monthly rentals in July was bigger than in June. Your objective will be to predict this new column.


In [126]:
datam =  pd.merge(data_june, data_july, on='film_id', how='outer')

In [127]:
datam

,film_id,year_x,month_x,total_rent_x,title_x,rental_duration_x,rental_rate_x,special_features_x,year_y,month_y,total_rent_y,title_y,rental_duration_y,rental_rate_y,special_features_y
0,1,2005.0,6.0,3.0,ACADEMY DINOSAUR,6.0,0.99,"Deleted Scenes,Behind the Scenes",2005,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,3,2005.0,6.0,1.0,ADAPTATION HOLES,7.0,2.99,"Trailers,Deleted Scenes",2005,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
2,4,2005.0,6.0,2.0,AFFAIR PREJUDICE,5.0,2.99,"Commentaries,Behind the Scenes",2005,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
3,5,2005.0,6.0,2.0,AFRICAN EGG,6.0,2.99,Deleted Scenes,2005,7,5,AFRICAN EGG,6,2.99,Deleted Scenes
4,6,2005.0,6.0,3.0,AGENT TRUMAN,3.0,2.99,Deleted Scenes,2005,7,7,AGENT TRUMAN,3,2.99,Deleted Scenes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,2,TRAFFIC HOBBIT,5,4.99,"Trailers,Commentaries"
954,904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,1,TRAIN BUNCH,3,4.99,"Trailers,Deleted Scenes"
955,913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,7,TROOPERS METAL,3,0.99,"Deleted Scenes,Behind the Scenes"
956,939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,3,VERTIGO NORTHWEST,4,2.99,"Commentaries,Behind the Scenes"


In [128]:
datam.rename(columns={'month_x': 'june', 'total_rent_x': 'total_rent_june','month_y': 'july', 'total_rent_y': 'total_rent_july','rental_duration_y':'rental_duration','title_y':'title','rental_rate_y':'rental_rate','special_features':'special_features' }, inplace=True)

In [129]:
datam

,film_id,year_x,june,total_rent_june,title_x,rental_duration_x,rental_rate_x,special_features_x,year_y,july,total_rent_july,title,rental_duration,rental_rate,special_features_y
0,1,2005.0,6.0,3.0,ACADEMY DINOSAUR,6.0,0.99,"Deleted Scenes,Behind the Scenes",2005,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,3,2005.0,6.0,1.0,ADAPTATION HOLES,7.0,2.99,"Trailers,Deleted Scenes",2005,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
2,4,2005.0,6.0,2.0,AFFAIR PREJUDICE,5.0,2.99,"Commentaries,Behind the Scenes",2005,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
3,5,2005.0,6.0,2.0,AFRICAN EGG,6.0,2.99,Deleted Scenes,2005,7,5,AFRICAN EGG,6,2.99,Deleted Scenes
4,6,2005.0,6.0,3.0,AGENT TRUMAN,3.0,2.99,Deleted Scenes,2005,7,7,AGENT TRUMAN,3,2.99,Deleted Scenes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,2,TRAFFIC HOBBIT,5,4.99,"Trailers,Commentaries"
954,904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,1,TRAIN BUNCH,3,4.99,"Trailers,Deleted Scenes"
955,913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,7,TROOPERS METAL,3,0.99,"Deleted Scenes,Behind the Scenes"
956,939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005,7,3,VERTIGO NORTHWEST,4,2.99,"Commentaries,Behind the Scenes"


In [130]:
datam = datam.drop(['year_x', 'year_y','title_x','rental_duration_x', 'rental_rate_x', 'special_features_x'], axis=1)

In [131]:
datam

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y
0,1,6.0,3.0,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,3,6.0,1.0,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
2,4,6.0,2.0,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
3,5,6.0,2.0,7,5,AFRICAN EGG,6,2.99,Deleted Scenes
4,6,6.0,3.0,7,7,AGENT TRUMAN,3,2.99,Deleted Scenes
...,...,...,...,...,...,...,...,...,...
953,903,NaN,NaN,7,2,TRAFFIC HOBBIT,5,4.99,"Trailers,Commentaries"
954,904,NaN,NaN,7,1,TRAIN BUNCH,3,4.99,"Trailers,Deleted Scenes"
955,913,NaN,NaN,7,7,TROOPERS METAL,3,0.99,"Deleted Scenes,Behind the Scenes"
956,939,NaN,NaN,7,3,VERTIGO NORTHWEST,4,2.99,"Commentaries,Behind the Scenes"


In [132]:
# handle missing values, should be replace by 6 in the month and cero in total rent
datam['june'].isnull().sum()

58

In [133]:
datam['june'].fillna(6.0)

0      6.0
1      6.0
2      6.0
3      6.0
4      6.0
      ... 
953    6.0
954    6.0
955    6.0
956    6.0
957    6.0
Name: june, Length: 958, dtype: float64

In [134]:
datam['june'] = datam['june'].fillna(7)

In [135]:
datam['june'].isnull().sum()

0

In [136]:
datam['july'].isnull().sum()

0

In [137]:
datam['total_rent_june'].isnull().sum()

58

In [138]:
datam['total_rent_june'] = datam['total_rent_june'].fillna(0)

In [139]:
datam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958 entries, 0 to 957
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   film_id             958 non-null    int64  
 1   june                958 non-null    float64
 2   total_rent_june     958 non-null    float64
 3   july                958 non-null    int64  
 4   total_rent_july     958 non-null    int64  
 5   title               958 non-null    object 
 6   rental_duration     958 non-null    int64  
 7   rental_rate         958 non-null    float64
 8   special_features_y  958 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 74.8+ KB


In [140]:
datam['total_rent_june'] = datam['total_rent_june'].astype('int')

In [141]:
datam.head()
#good :)

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
2,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
3,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes
4,6,6.0,3,7,7,AGENT TRUMAN,3,2.99,Deleted Scenes


In [142]:
datam = datam.sort_values(by = 'film_id').reset_index(drop=True)

In [143]:
datam

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes"
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes"
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes"
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes"
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes
...,...,...,...,...,...,...,...,...,...
953,996,6.0,1,7,3,YOUNG LANGUAGE,6,0.99,"Trailers,Behind the Scenes"
954,997,6.0,1,7,2,YOUTH KICK,4,0.99,"Trailers,Behind the Scenes"
955,998,6.0,2,7,3,ZHIVAGO CORE,6,0.99,Deleted Scenes
956,999,6.0,2,7,7,ZOOLANDER FICTION,5,2.99,"Trailers,Deleted Scenes"


In [144]:
datam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   film_id             958 non-null    int64  
 1   june                958 non-null    float64
 2   total_rent_june     958 non-null    int64  
 3   july                958 non-null    int64  
 4   total_rent_july     958 non-null    int64  
 5   title               958 non-null    object 
 6   rental_duration     958 non-null    int64  
 7   rental_rate         958 non-null    float64
 8   special_features_y  958 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 67.5+ KB


In [145]:
def bigger_june(x):
    if x['total_rent_june'] < x['total_rent_july']:
        return 'Yes'
    else:
        return 'No'

datam['bigger_june'] = datam.apply(bigger_june,axis=1)

In [146]:
datam

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y,bigger_june
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes",Yes
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes",Yes
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes",Yes
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes",Yes
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes,Yes
...,...,...,...,...,...,...,...,...,...,...
953,996,6.0,1,7,3,YOUNG LANGUAGE,6,0.99,"Trailers,Behind the Scenes",Yes
954,997,6.0,1,7,2,YOUTH KICK,4,0.99,"Trailers,Behind the Scenes",Yes
955,998,6.0,2,7,3,ZHIVAGO CORE,6,0.99,Deleted Scenes,Yes
956,999,6.0,2,7,7,ZOOLANDER FICTION,5,2.99,"Trailers,Deleted Scenes",Yes


In [147]:
datam['bigger_june'].unique()

array(['Yes', 'No'], dtype=object)

In [148]:
datam['bigger_june'].value_counts()

Yes    952
No       6
Name: bigger_june, dtype: int64

### 5. Read the data into a Pandas dataframe.

In [149]:
datam

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y,bigger_june
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes",Yes
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes",Yes
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes",Yes
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes",Yes
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes,Yes
...,...,...,...,...,...,...,...,...,...,...
953,996,6.0,1,7,3,YOUNG LANGUAGE,6,0.99,"Trailers,Behind the Scenes",Yes
954,997,6.0,1,7,2,YOUTH KICK,4,0.99,"Trailers,Behind the Scenes",Yes
955,998,6.0,2,7,3,ZHIVAGO CORE,6,0.99,Deleted Scenes,Yes
956,999,6.0,2,7,7,ZOOLANDER FICTION,5,2.99,"Trailers,Deleted Scenes",Yes


### 6. Analyze extracted features and transform them. You may need to encode some categorical variables or scale numerical variables.


I will try to predict Yes/No with the special features. 

In [150]:
datam['special_features_y'].unique()

array(['Deleted Scenes,Behind the Scenes', 'Trailers,Deleted Scenes',
       'Commentaries,Behind the Scenes', 'Deleted Scenes', 'Trailers',
       'Commentaries,Deleted Scenes',
       'Trailers,Commentaries,Behind the Scenes', 'Trailers,Commentaries',
       'Trailers,Behind the Scenes',
       'Commentaries,Deleted Scenes,Behind the Scenes',
       'Trailers,Commentaries,Deleted Scenes',
       'Trailers,Deleted Scenes,Behind the Scenes', 'Behind the Scenes',
       'Trailers,Commentaries,Deleted Scenes,Behind the Scenes',
       'Commentaries'], dtype=object)

could separate the special fueatures in four different yes/no columns: Deleted Scenes, Behind the Scenes, Trailers, Commentaries

In [151]:
def deleted_scenes (x):
    if 'Deleted Scenes' in x['special_features_y']:
        return 1
    else:
        return 0
        
datam['SF_deleted_scenes'] = datam.apply(deleted_scenes,axis=1)

In [152]:
datam.head()

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y,bigger_june,SF_deleted_scenes
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes",Yes,1
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes",Yes,1
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes",Yes,1
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes",Yes,0
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes,Yes,1


In [153]:
def Trailers (x):
    if 'Trailers' in x['special_features_y']:
        return 1
    else:
        return 0
        
datam['SF_trailers'] = datam.apply(Trailers,axis=1)
datam.head()

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y,bigger_june,SF_deleted_scenes,SF_trailers
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes",Yes,1,0
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes",Yes,1,1
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes",Yes,1,1
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes",Yes,0,0
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes,Yes,1,0


In [154]:
def Behind_the_Scenes (x):
    if 'Behind the Scenes' in x['special_features_y']:
        return 1
    else:
        return 0
        
datam['SF_behind_the_scenes'] = datam.apply(Behind_the_Scenes,axis=1)
datam.head()

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y,bigger_june,SF_deleted_scenes,SF_trailers,SF_behind_the_scenes
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes",Yes,1,0,1
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes",Yes,1,1,0
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes",Yes,1,1,0
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes",Yes,0,0,1
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes,Yes,1,0,0


In [155]:
def Commentaries (x):
    if 'Commentaries' in x['special_features_y']:
        return 1
    else:
        return 0
        
datam['SF_commentaries'] = datam.apply(Commentaries,axis=1)
datam.head()

,film_id,june,total_rent_june,july,total_rent_july,title,rental_duration,rental_rate,special_features_y,bigger_june,SF_deleted_scenes,SF_trailers,SF_behind_the_scenes,SF_commentaries
0,1,6.0,3,7,9,ACADEMY DINOSAUR,6,0.99,"Deleted Scenes,Behind the Scenes",Yes,1,0,1,0
1,2,7.0,0,7,2,ACE GOLDFINGER,3,4.99,"Trailers,Deleted Scenes",Yes,1,1,0,0
2,3,6.0,1,7,4,ADAPTATION HOLES,7,2.99,"Trailers,Deleted Scenes",Yes,1,1,0,0
3,4,6.0,2,7,12,AFFAIR PREJUDICE,5,2.99,"Commentaries,Behind the Scenes",Yes,0,0,1,1
4,5,6.0,2,7,5,AFRICAN EGG,6,2.99,Deleted Scenes,Yes,1,0,0,0


In [156]:
datam.columns

datasf = datam[['film_id','bigger_june','SF_deleted_scenes','SF_trailers','SF_behind_the_scenes','SF_commentaries']]
datasf.head()

,film_id,bigger_june,SF_deleted_scenes,SF_trailers,SF_behind_the_scenes,SF_commentaries
0,1,Yes,1,0,1,0
1,2,Yes,1,1,0,0
2,3,Yes,1,1,0,0
3,4,Yes,0,0,1,1
4,5,Yes,1,0,0,0


In [157]:
def Yes_No_bin (x):
    if x['bigger_june'] == 'Yes':
        return 1
    else:
        return 0
        
datasf['bigger_june'] = datasf.apply(Yes_No_bin,axis=1)
datasf.head()

/var/folders/2b/6f80d7g11sl53b_wrqy2w81r0000gn/T/ipykernel_60273/2535336393.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasf['bigger_june'] = datasf.apply(Yes_No_bin,axis=1)


,film_id,bigger_june,SF_deleted_scenes,SF_trailers,SF_behind_the_scenes,SF_commentaries
0,1,1,1,0,1,0
1,2,1,1,1,0,0
2,3,1,1,1,0,0
3,4,1,0,0,1,1
4,5,1,1,0,0,0


### 7. Create a logistic regression model to predict this new column from the cleaned data. 

In [158]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [160]:
X = datasf[['SF_deleted_scenes','SF_trailers','SF_behind_the_scenes','SF_commentaries']]
X.head()

,SF_deleted_scenes,SF_trailers,SF_behind_the_scenes,SF_commentaries
0,1,0,1,0
1,1,1,0,0
2,1,1,0,0
3,0,0,1,1
4,1,0,0,0


In [161]:
y = datasf['bigger_june']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: bigger_june, dtype: int64

In [162]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.3)

In [163]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [164]:
model.fit(X_train, y_train)

LogisticRegression()

In [165]:
model.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### 8.Evaluate the results.

Because the Data have moslty 'Yes' answer (99,37%) for y, the model will be very good at predicting 'Yes' and really bad at predicting 'No', so is natural that the model predict just 1 ('Yes') values. I could try others variables but probably the outcome will be the same. 